In [1]:
import os
import numpy
import sys

In [2]:
os.getcwd()

'/home/wzz/Projects/PyVDrive/prototypes'

In [45]:
def scan_log_file(log_file_name, block_start_flag):
        """
        Task list:
        1. enable 'return' button
        2. scan for summary including
          a) number of blocks
          b) size of each block
          c) ... ...
        :return:
        """
        # check
        # ...
        # ... 
        
        sum_dict = dict()

        # set up summary parameters
        buffer_size = 6
        buffer_lines = list()

        # open file and search block starter
        block_key = None
        last_block_key = None
        block_line_list = None
        num_lines_recorded = 0

        with open(log_file_name, 'r') as log_file:
            for line_number, line in enumerate(log_file):
                # parse the line
                line = line.strip()
                
                # fill buffer
                buffer_lines.append(line)
                if len(buffer_lines) > buffer_size:
                    buffer_lines.pop(0)
                      
                # check the line
                if block_key is not None:
                    # in a recording stage
                    num_lines_recorded += 1
                    # check quit condition
                    if num_lines_recorded == buffer_size:
                        # finished recording job
                        # record
                        sum_dict[block_key] = buffer_lines[:]
                        # reset block key
                        last_block_key = block_key
                        block_key = None
                        num_lines_recorded = 0

                elif line.startswith(block_start_flag):
                    # not in recording stage but it is a start of a block
                    block_key = line_number
                    num_lines_recorded = 1
                    if last_block_key is not None:
                         sum_dict[last_block_key].extend(buffer_lines[:-1])
                # END-IF
            # END-FOR
        # END-WITH
        
        # list lines
        if block_key is None:
            block_key = last_block_key
            if block_key is None:
                raise ValueError
        if block_key in sum_dict:
            sum_dict[block_key].extend(buffer_lines[:])
        else:
            sum_dict[block_key] = buffer_lines[:]

        # prepare the summary
        """
        sum_str = ''
        # sum_str += 'comments: \n'
        sum_str += '    %s\n\n' % self._comments
        for block_index, block_start_line_number in enumerate(self._blockLineDict.keys()):
            sum_str += 'block %d\n' % block_index
            for index, line in enumerate(self._blockLineDict[block_start_line_number]):
                sum_str += '%-10d: %s\n' % (index+block_start_line_number,
                                            line.strip())
            sum_str += '\n'
            # END-FOR

        # set to summary view
        self.ui.plainTextEdit_summaryView.setPlainText(sum_str)
        """

        return sum_dict

In [56]:
# %%timeit

log_file_name = '/SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample1_again3/specimen.dat'      #    697117 100 loops, best of 3: 4.92 ms per loop
#log_file_name = '/SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample9_continue/specimen9_5.dat' #    605232 May  1 11:53  80ms/4.4ms
#log_file_name = '/SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample9_continue/sample8_1.dat'   #  26467874 May  1 11:40
#log_file_name = '/SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample1_4/specimen.dat'           # 265849517 May  1 09:05
log_file_name = '/SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample3/specimen.dat'              # 1.2GB... perfect example
file_size = os.path.getsize(log_file_name)
file_size_mb = file_size/1024/1024
print file_size, type(file_size)
print 'File size = %d MB' % file_size_mb
block_start_flag = 'Data Acquisition'

sum_dict = scan_log_file(log_file_name, block_start_flag)
for block_key in sorted(sum_dict.keys()):
    print 'Block %d' % block_key
    for line in sum_dict[block_key]:
        print '\t', line

1312050065 <type 'int'>
File size = 1251 MB
Block 2
	Data Acquisition						Time:	19.504395	s	4/27/2016 18:20:34 PM
	Time	X Matrix Displacement	X Matrix Force	X Matrix Strain	X Matrix Stress
	s	mm	N	mm/mm	N/mm�
	11.501465	6.8169622	49.618851	-0.00041163471	6.5720334
	11.503418	6.8167315	49.263912	-0.00041204781	6.5250216
	11.505371	6.8170824	49.543495	-0.00041292526	6.5620522
	308.3999	6.8297586	250.6969	-0.00032636127	33.204887
	308.40186	6.829587	250.55898	-0.00032632414	33.186619
	308.40381	6.8296928	250.60588	-0.00032717222	33.192833
	308.40576	6.8294964	250.4288	-0.00032722391	33.169376
	
Block 152022
	Data Acquisition						Time:	12.76123	s	4/27/2016 18:30:25 PM
	Time	X Matrix Displacement	X Matrix Force	X Matrix Strain	X Matrix Stress
	s	mm	N	mm/mm	N/mm�
	4.7587891	6.8301091	249.81729	-0.00032187041	33.088383
	4.7602539	6.8300123	249.79567	-0.00032172498	33.085518
	4.7626953	6.8296976	250.27213	-0.00032125309	33.148624
	27.12207	6.829761	250.22838	-0.00032231255	33.14283
	27.12402

* /SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample9_continue/sample8_1.dat:

  The slowest run took 5.77 times longer than the fastest. This could mean that an intermediate result is being cached.
  1 loop, best of 3: 137 ms per loop
  
  
* /SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample1_4/specimen.dat

  1 loop, best of 3: 1.42 s per loop


In [18]:
log_file_name = '/SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample1_4/specimen.dat'    
print log_file_name
size = os.path.getsize(log_file_name)
print size, type(size), size/1024/1024

/SNS/VULCAN/IPTS-14094/shared/IPTS_14094_sample1_4/specimen.dat
265849517 <type 'int'> 253
